# Get Started

> ## Download the data

> Write a small function to get the data. If the data changes constantly you can call this function perodically and get the new and updated data. The `fetch_housing_data()` downloads the data, uncompresses the data.

In [1]:
import os
import tarfile
from six.moves import urllib

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [3]:
def fetch_housing_data(h_url=HOUSING_URL, h_path=HOUSING_PATH):
    if not os.path.isdir(h_path):
        os.makedirs(h_path)

    tgz_path = os.path.join(h_path, "housing.tgz")
    urllib.request.urlretrieve(h_url, tgz_path)
    h_tgz = tarfile.open(tgz_path)
    h_tgz.extractall(path=h_path)
    h_tgz.close()

In [4]:
fetch_housing_data()

> ## Working with **Pandas, matplotlib, train_test_split from sklearn**

Here we try to achieve the following:
* gain insights from the data through the **Pandas** library.
* plot and gain insights from the `hist()` plots.
* create a test set free from biases.

In [ ]:
import pandas as pd

def load_housing_data(h_path=HOUSING_PATH):
    csv_path = os.path.join(h_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
data = load_housing_data()

* `data.head()` shows the first 5 elements
* `data.tail()` shows the last 5 elements

In [ ]:
data.head()

* `data.info()` gives the info of each column in the data

> From the `info` we see that only $ocean\_proximity$ has a **non-numerical** **Dtype**, this column contains an **object**, thus this column can contain any valid python object

In [ ]:
data.info()

* `data[col].value_counts()` gives the count of each categorical data in a particular coulmn.



In [ ]:
data["ocean_proximity"].value_counts()

* `data.describe()` gives some important info regarding the numerical values in the data

* **Note**: The null values are ignored thus the count of $total\_bedrooms$ is *20433* and not *20640* as there are *207* missing values

* The  **std**
row  shows  the  *standard  deviation*,  which  measures  how  dispersed  the  values  are.

* The  *25%,  50%,  and  75%*  rows  show  the  corresponding  percentiles:  a  percentile  indi‐
cates the value below which a given percentage of observations in a group of observa‐
tions  falls. **eg.** For  example,  25%  of  the  districts  have  a  housing_median_age  lower  than
18, while 50% are lower than 29 and 75% are lower than 37. These are often called the
25th percentile (or 1st quartile), the median, and the 75th percentile (or 3rd quartile).

In [ ]:
data.describe()

* Another way to make sense of the data is to plot the data and we can use the histogram plot for the same
* A *histogram* shows the number of insatances (on the vertical axis) that have a given range on the horizontal axis

> **Note:** You can either plot this obe attribute at a time, or you can call `hist()` method in the whole dataset

In [ ]:
import matplotlib.pyplot as plt
data.hist(bins=50, figsize=(20, 15))
plt.show()

> Creating a test set

* Using the standard method

In [ ]:
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data)*test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]


In [ ]:
train_set, test_set = split_train_test(data, 0.2)
print(len(train_set), len(test_set))

* Now in the above case of spliting the data every time we run the cell we will end up having a different test and train set. eventually out model has ended up seeing the entire data. To solve this issue we can use the `np.random.seed()` before `np.random.permutation()` and we will end up getting the same data.
* But, now say our data is constantly being updated. What happens is every time we take a spilt we end up having some previous test data in the train set and vice versa.
* To have a stable train/test set even after updating the dataset, a common solution is to use an instance identifier to decide whether or not it should be in the test set. For example, you could compute a hash of each instance's identifier and put that instance in the tset set if the has is lower that or equal to 20% of the minimum hash value. This ensures that ths test set will remain consistant across multiple runs


    
        from zlip import crc32
        def test_set_check(indentifier, test_size):
            return crc32(np.int64(indentifier)) & 0xffffffff < test_ratio*2**32
        
        def spilt_train_test_by_id(data, test_ratio, id_column):
            ids = data[column_id]
            in_test_set = ids.apply(lambda id_:test_set_check(id_, test_ratio))
            return data.loc[~in_test_set], data.loc[in_test_set]

> You can also use the `train_test_split()` function from the *sklearn* module imported as `from sklearn.model_selection import train_test_split()`

* So far we have considered purely random sampling methods. This is generally fine if
your dataset is large enough (especially relative to the number of attributes), but if it
is  not,  you  run  the  risk  of  introducing  a  significant  sampling  bias.

* When  a  survey
company decides to call 1,000 people to ask them a few questions, they don’t just pick
1,000  people  randomly  in  a  phone  book.  They  try  to  ensure  that  these  1,000  people
are  representative  of  the  whole  population.  For  example,  the  US  population  is  com‐
posed of 51.3% female and 48.7% male, so a well-conducted survey in the US would
try to maintain this ratio in the sample: 513 female and 487 male. This is called strati‐
fied  *sampling*: the  population  is  divided  into  homogeneous  subgroups  called  *strata*,
and the right number of instances is sampled from each stratum to guarantee that the
test  set  is  representative  of  the  overall  population.  

* If  they  used  purely  random  sam‐
pling, there would be about 12% chance of sampling a skewed test set with either less
than  49%  female  or  more  than  54%  female.  Either  way,  the  survey  results  would  be
significantly biased.


* Suppose  you  chatted  with  experts  who  told  you  that  the  median  income  is  a  very
important  attribute  to  predict  median  housing  prices.  You  may  want  to  ensure  that
the test set is representative of the various categories of incomes in the whole dataset.

> Let's look at the histogram of the meadian income

* The  following  code  uses  the
[pd.cut()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html?highlight=cut#pandas.cut) function  to  create  an  income  category  attribute  with  5  categories  (labeled
from 1 to 5): category 1 ranges from 0 to 1.5 (i.e., less than $15,000), category 2 from 1.5 to 3, and so on:

In [ ]:
data["income_cat"] = pd.cut(data["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
data

In [ ]:
data["income_cat"].hist()

* Now  you  are  ready  to  do  stratified  sampling  based  on  the  income  category.  For  this
you can use Scikit-Learn’s `StratifiedShuffleSplit` class:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state=42)

for train_idx, test_idx in split.split(data, data["income_cat"]):
    strat_train_set = data.iloc[train_idx]
    strat_test_set = data.iloc[test_idx]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

> Now remove the `income_cat` attribute 

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Visialize the data to gain insights

> Make a copy of the train set

In [ ]:
visual_cp = strat_train_set.copy()

[visual_cp.plot()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.html?highlight=plot#pandas.Series.plot)

In [ ]:
visual_cp.plot(kind='scatter', x='longitude', y='latitude')

> Set alpha patameter to 0.1 for better visualization
Setting alpha to 0.1 helps visualize place where there is high density of data points

In [ ]:
visual_cp.plot(kind='scatter', x = 'longitude', y='latitude', alpha=0.1)

> Plot the median_house_price

* The radius of each circle represents
the district’s population (option s), and the color represents the price (option c). We
will  use  a  predefined  color  map  (option  cmap)  called  jet,  which  ranges  from  blue
(low values) to red (high prices)

In [ ]:
visual_cp.plot(kind='scatter', x='longitude', y = 'latitude', alpha=0.4, s=visual_cp["population"]/100, label='population', figsize=(10,7),c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)

The plot tells us that the housing prices are very much related to the location (eg. close to the ocean) and tp the population density

> ## Looking for correlations

* [Covariance in Statistics](https://www.youtube.com/watch?v=mlxwAPTjiCA)
* [Statistics- What is Pearson Correlation Coefficient? Difference between Correlation and Covariance](https://www.youtube.com/watch?v=6fUYt1alA1U)
* [Spearman's rank correlation coefficient- Statistics](https://www.youtube.com/watch?v=CIQ3u7CvOEU)

> you can easily compute the *standard correlation coefficient* also called *Pearson's r* between every pair using the `corr()` method

In [ ]:
corr_matrix = visual_cp.corr()

In [ ]:
corr_matrix

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

* The correlation coefficient ranges from -1 to 1
* When it is close to 1, it means that there is a strong positive correlation; *for eg*, *meadian house* value tends to go up when the median income goes up.
* When the coefficient is close to -1, it means that there is a strong negative correlation;
you see a strong negative correlation between *latitude* and the meadian of the house value.
* Finally the coefficient close to 0 mean that there os no linear correlation 

> **🚀Note:** The correlation coefficient only measures linear correlations(*if x goes up, then generally y goes up/down*). It may completely miss out on non-linear relationships. For example, your height in inches has a correla‐
tion coefficient of 1 with your height in feet or in nanometers.

> 🤖 **Another way** to check for correlations between attributes is to use Pandas method `scatter_matrix`, which plots every numerical attribute against every other numerical attribute.

> **🚀Note:** Since there are now 11 numerical attributes, you would get 11<sup>2</sup> =
121  plots


In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(visual_cp[attributes], figsize=(12,8))

> The main diagonal (top left to bottom right) would be full of straight lines if Pandas
plotted each variable against itself, which would not be very useful. So instead Pandas
displays a histogram of each attribute (other options are available; see Pandas’ docu‐
mentation for more details).

In [ ]:
visual_cp.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

> 💡 This  plot  reveals  a  few  things.  First,  the  correlation  is  indeed  very  strong;  you  can
clearly  see  the  upward  trend  and  the  points  are  not  too  dispersed.  Second,  the  price
cap  that  we  noticed  earlier  is  clearly  visible  as  a  horizontal  line  at  \$500,000.  But  this
plot  reveals  other  less  obvious  straight  lines:  a  horizontal  line  around  \$450,000,
another around \$350,000, perhaps one around \$280,000, and a few more below that.
You may want to try removing the corresponding districts to prevent your algorithms
from learning to reproduce these data quirks.

The total number of romms in a district is not very useful if you don't know how many households there are. similarly number of bedrooms by itself is not useful

In [ ]:
visual_cp["rooms_per_household"] = visual_cp["total_rooms"]/visual_cp["households"]
visual_cp["bedrooms_per_room"] = visual_cp["total_bedrooms"]/visual_cp["total_rooms"]
visual_cp["population_per_household"] = visual_cp["population"]/visual_cp["households"]

In [ ]:
corr_mat_2 = visual_cp.corr()

In [ ]:
corr_mat_2["median_house_value"].sort_values(ascending=False)

# Preparing data for Machine Learning Algorithms

> 🧠But first let’s revert to a clean training set (by copying strat_train_set once again),
and let’s separate the predictors and the labels since we don’t necessarily want to apply
the  same  transformations  to  the  predictors  and  the  target  values  (note  that  drop()  
creates a copy of the data and does not affect strat_train_set)

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

> ### Handling missing data

> 🤖 You have three options to fix missing data:
* Get rid of the corresponding districts. `housing.dropna(subset=["total_bedrooms"])`
* Get rid of the whole arribute. `housing.drop("total_bedrooms", axis=1)`
* Set the values to some value(zero, the mean, the median, etc)
        median = housing["total_bedrooms"].median()
        housing["total_bedrooms"].fillna(median, inplace=True)
if you choose option 3 then save the median and for now update only the test set and later of used the saved median to update the missing values in the test set.

handling missing data using sklearns `SimpleImputer` class.
Using the Simple Imputer we can apply any startegy we want like, mean, median, etc.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

In [ ]:
# removing ocean_proximity because SimpleImputer can be used only with numerical data
housing_num = housing.drop("ocean_proximity", axis = 1)

In [ ]:
imputer.fit(housing_num)

The imputer has simply computed the median of each attribute and stored the result in its `statistics_` instance variable. The `imputer.fit()` computes the `strategy` for all the attributes in the data and replaces any missing value with the corresponding `strategy`

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median()

In [ ]:
#transform the data
X = imputer.transform(housing_num)

In [ ]:
X

In [ ]:
#convert X into pd dataframe
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
housing_tr

### Handling text and categorical values

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

we use sklearn's `OrdinalEncoder` to convert the categorical text data into numerical equivalent numbers

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordial_encoder = OrdinalEncoder()
housing_cat_encoded = ordial_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

to see all the categories that were present us `categories`

In [ ]:
ordial_encoder.categories_

> One hot Encoding

to perform one-hot encoding use the `OneHotEncoder` class from sklearn

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_1_cat = cat_encoder.fit_transform(housing_cat)
housing_1_cat

> 💡 Notice that the output is scipy sparse matrix, instead of Numpy array. This is useful if there are 1000 categories full of 1's and 0's except for a single 1 per row, it is wasteful of memory. This this shows only the places where 1 is stored. But if you want a 2D numpy array then use the `toarray()` method.

In [ ]:
housing_1_cat.toarray()

You can also pass text value for this encoder and you can see all the categories in the `categories_` attribute

In [ ]:
cat_encoder.categories_

###Custom Transforms

You need to create a class and implement 3 methods `fit() returning self`, `transform()`, `fit_transform()` you can get rid fo the last one if you add `TransformerMixin` for the base class. If you add `BaseEstimator` as a base class and avoid $*agrs$ and $**kargs$ in your custom constructor then you will get two additional methods `get_params()` and `set_params()` which will be useful for hyperparameter tuning. The following custom transformer adds the combined attributes discussed earlier,

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

### Building Transformation pipelines

We can build a transformation pipeline using sklearn's `Pipeline` class

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

The Pipeline constructor takes a list of name/estimator pairs defining the sequence of steps. All but last estimator must be transformers(i.e. they must have the `fit_transform()` method)

Sklearn's ColumnTransformer class is used to apply these transforms to each and every transforms that are available in the same order.

In the constructor it takes a list of tuples, where each tuple contains a name a transformer, and a list of names of columns that the transformer should be applied to.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
                                   ("num", num_pipeline, num_attribs),
                                   ("cat", OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
num_attribs

In [ ]:
housing_prepared

# Select and train a Model

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions: ", lin_reg.predict(some_data_prepared))
print("labels: ", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

##Better Evaluation Using Cross-Validation

You should not touch the test data until you are ready for production instead you should use a cross validation test for better evaluation.

A great alternative is to use sklearns k-fold cross-validation feature. The following code randomly splits the training set into 10 subsets called folds, then it trains and evaluates the Decision Tree model 10 times, picking a different fold for evalutaion every time and training on the other 9 folds. The result is an array containing the 10 evaluation scores

> 💡 Note: Sklearn's cross-validation feature expects a **utility function (grater the better)** rather than a **cost function (lower the better)**, so the scoring function is actually the opposite of the MSE (i.e. a negative value), which is why the preceding code computes `-scores` before calcaulting the root.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

From the above scores we can conclude that the decision tree is overfitting the model badly and the performance of linear regression is better than decision tree

## Random Forest

Random Forest works by training many decision trees in random subset of the features, the averaging out there predictions. Building a model on top of many other models is called *Ensemble Learning*, and it is often a great way to push ML algos.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
print('rmse: ', forest_rmse)
display_scores(forest_rmse_scores)

# Fine Tune Your models

One option to change hyperparameter tuning is to change them mannually, until you find a good combination. but its too time comsuming.

Instead using sklearn `GridSearchCV`. All you need to do is tell which hyperparemeters you wnat it to experiment with and what values to try out, and it will use cross-validation to evaluate all the possible combination of hyperparameter values

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
              {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
              {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

This `param_grid` tells sklearn to first evaluate all the 3x4=12 combinations of `n_estimators` and `max_features` hyperparameter values specified in the first `dict`, then try all 2x3=6combinations of hyperparameter values in the secont `dict`, but this time with the `bootstrap` set to `False` which is `True` by default.

The gird search will explore 12+6=18 combinations of `RandomForestRegresoor`, it will train the model 5 times (since we are using five-fold cross validation). Meaning it will train 18x5=90 rounds of training

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

>💡 Note: If `GridSeachCV` is initialized with `refit=True` (which is the default), the once it finds the best estimator using coress-validation, it retrains it in the whole training set. This is usually a good ideam since feeding it more data will likely imporve its performance.

In [ ]:
cvres = grid_search.cv_results_

In [ ]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

Similar to `GridSearchCV` you can use `RandomizedSearchCv`, but instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration.